# Project 3: Customers Served By Target BART Lines

University of California, Berkeley

Master of Information and Data Science (MIDS) program

w205 - Fundamentals of Data Engineering


# Included Modules and Packages

Code cell containing your includes for modules and packages

Some starter code is provided

You may change the starter code as needed

You may add as much code and/or as many code cells as you need

In [36]:
import csv

import math
import numpy as np
import pandas as pd

import psycopg2

from geographiclib.geodesic import Geodesic

# Supporting code

Code cells containing any supporting code, such as connecting to the database, any functions, etc.  

Remember you can freely use any code from the labs. You do not need to cite code from the labs.

Some starter code is provided

You may change the starter code as needed

You may add as much code and/or as many code cells as you need

In [37]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [38]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [39]:
cursor = connection.cursor()

# Query the list of stations on the Red and Orange Lines

1. Leverage the functions from 3.5 to returns zip codes within X miles of a station.

In [40]:
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

In [41]:
def my_station_get_zips(station, miles):
    "given a station, pull all zip codes with miles distance, print them, sum the population"
    
    connection.rollback()
    
    query = "select latitude, longitude from stations "
    query += "where station = '" + station + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select zip, population from zip_codes "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by 1 "

    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    

    query_result = ""
    
    for row in rows:
        zip = row[0]
        population = row[1]
        query_result += "('" + station + "_" + str(zip) + "','" + station + "'," +  str(population) + ", '" + str(zip) + "'),\n"
        
    return query_result

2. Drop the tables if they exists already and then create it

In [42]:
connection.rollback()

query = """

drop table if exists stage_station_zips;

create table stage_station_zips (
  id varchar(106),
  station varchar(100),
  population numeric(7),
  zip varchar(5)
);

drop table if exists stage_station_zips_yellowexpansion;

create table stage_station_zips_yellowexpansion (
  id varchar(106),
  station varchar(100),
  population numeric(7),
  zip varchar(5)
);


drop table if exists station_zips_customers;
drop table if exists station_zips_customers_yellowexpansion;


"""

cursor.execute(query)

connection.commit()

3. Get the list of all stations on the red and orange lines (the lines directly connected to the Berkeley store) and create a table of zip codes that are within 1 mile of the station. Also explore what it would look like it we expanded to the yellow line in the future.

In [43]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select station
from lines
where line in ('red','orange')

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station
0,Richmond
1,El Cerrito del Norte
2,El Cerrito Plaza
3,North Berkeley
4,Downtown Berkeley
5,Ashby
6,MacArthur
7,19th Street
8,12th Street
9,Lake Merritt


In [44]:
query = """

select station
from lines
where line in ('red','orange')


"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

insert_query = """

INSERT INTO 
    stage_station_zips (id, station, population, zip)
VALUES
"""

for row in rows:
    insert_query += my_station_get_zips(row[0],1)
    
    
insert_query = insert_query[:-2] + ";"

#print(insert_query)

cursor.execute(insert_query)

connection.commit()

In [45]:
query = """

select station
from lines
where line in ('red','orange','yellow')


"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

insert_query = """

INSERT INTO 
    stage_station_zips_yellowexpansion (id, station, population, zip)
VALUES
"""

for row in rows:
    insert_query += my_station_get_zips(row[0],1)
    
    
insert_query = insert_query[:-2] + ";"

#print(insert_query)

cursor.execute(insert_query)

connection.commit()

4. Validate the results in the station_zips table. 

Notes: 
1. The table will contain duplicate values at stations where red and orange intersect (using distinct to eliminate)

5. Join the customers table to the station_zips table to create a table of all customers that are within 1 mile of the red and orange lines.

In [46]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select distinct on (id) *
from stage_station_zips
order by id

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,id,station,population,zip
0,12th Street_94612,12th Street,16062,94612
1,16th Street Mission_94103,16th Street Mission,28735,94103
2,16th Street Mission_94114,16th Street Mission,34754,94114
3,19th Street_94612,19th Street,16062,94612
4,24th Street Mission_94110,24th Street Mission,74161,94110
5,24th Street Mission_94114,24th Street Mission,34754,94114
6,Ashby_94702,Ashby,17092,94702
7,Ashby_94703,Ashby,21937,94703
8,Ashby_94704,Ashby,29190,94704
9,Balboa Park_94112,Balboa Park,85166,94112


Also check the yellow line expansion data:

In [47]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select distinct on (id) *
from stage_station_zips_yellowexpansion
order by id

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,id,station,population,zip
0,12th Street_94612,12th Street,16062,94612
1,16th Street Mission_94103,16th Street Mission,28735,94103
2,16th Street Mission_94114,16th Street Mission,34754,94114
3,19th Street_94612,19th Street,16062,94612
4,24th Street Mission_94110,24th Street Mission,74161,94110
...,...,...,...,...
61,SFO_94128,SFO,135,94128
62,South Hayward_94544,South Hayward,79235,94544
63,Union City_94587,Union City,74601,94587
64,Walnut Creek_94597,Walnut Creek,22734,94597


In [48]:

query = """

with s as (
select distinct on (id) *
from stage_station_zips
)
select
    s.station,
    s.zip,
    c.customer_id,
    c.street
into station_zips_customers
from s
join customers as c
    on s.zip = c.zip

"""

cursor.execute(query)

connection.commit()

Repeat for the yellow line expansion proposal:

In [49]:
query = """

with s as (
select distinct on (id) *
from stage_station_zips_yellowexpansion
)
select
    s.station,
    s.zip,
    c.customer_id,
    c.street
into station_zips_customers_yellowexpansion
from s
join customers as c
    on s.zip = c.zip

"""

cursor.execute(query)

connection.commit()

6. Verify the results in the new table and download the file to csv

In [50]:
connection.rollback()
    
query = """
    
copy (select *
    from station_zips_customers
    order by customer_id)
to '/user/projects/project-3-mmartin131/code/station_zips_customers.csv' delimiter ',' NULL '' csv header;

copy (select *
    from stage_station_zips
    order by id)
to '/user/projects/project-3-mmartin131/code/station_zips.csv' delimiter ',' NULL '' csv header;





copy (select *
    from station_zips_customers_yellowexpansion
    order by customer_id)
to '/user/projects/project-3-mmartin131/code/station_zips_customers_yellowexpansion.csv' delimiter ',' NULL '' csv header;

copy (select *
    from stage_station_zips_yellowexpansion
    order by id)
to '/user/projects/project-3-mmartin131/code/station_zips_yellowexpansion.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)
    
connection.commit()

In [23]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from station_zips_customers
order by customer_id
"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,zip,customer_id,street
0,MacArthur,94609,1,5 Ramsey Place
1,MacArthur,94609,2,6 Londonderry Plaza
2,MacArthur,94609,3,548 Mcguire Parkway
3,MacArthur,94609,4,99 Kennedy Court
4,MacArthur,94609,5,51 Mcbride Drive
...,...,...,...,...
4410,Fremont,94536,8123,21 Lakewood Circle
4411,Fremont,94536,8124,1 Walton Center
4412,Fremont,94536,8125,8 Eastwood Circle
4413,Fremont,94536,8126,8 Tomscot Terrace


Repeat for yellow expansion:

In [51]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from station_zips_customers_yellowexpansion
order by customer_id
"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,zip,customer_id,street
0,Rockridge,94609,1,5 Ramsey Place
1,MacArthur,94609,1,5 Ramsey Place
2,Rockridge,94609,2,6 Londonderry Plaza
3,MacArthur,94609,2,6 Londonderry Plaza
4,Rockridge,94609,3,548 Mcguire Parkway
...,...,...,...,...
5227,Fremont,94536,8123,21 Lakewood Circle
5228,Fremont,94536,8124,1 Walton Center
5229,Fremont,94536,8125,8 Eastwood Circle
5230,Fremont,94536,8126,8 Tomscot Terrace


7. Determine the number of customers not within a mile of any of these stations (missing from the results) and compare to the total number of customers with berkeley as their closest store

In [52]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select 
    count(c.customer_id)
from customers as c
left join station_zips_customers as s
    ON s.customer_id = c.customer_id
WHERE s.customer_id is null and c.closest_store_id = 1 


"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,count
0,5737


In [53]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select 
    count(c.customer_id)
from customers as c
WHERE  c.closest_store_id = 1 

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,count
0,8138


Check how much this improved if we added the yellow line:

In [54]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select 
    count(c.customer_id)
from customers as c
left join station_zips_customers_yellowexpansion as s
    ON s.customer_id = c.customer_id
WHERE s.customer_id is null and c.closest_store_id = 1 


"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,count
0,5149
